In [5]:
# Kør dette efter bash scriptet til at generere gff3 fil:

import os
import sys
import pandas as pd

In [63]:
# Åben gff3 fil

subtype="FN907962_1"
maintype = "HPV11"

gffFolder = "/home/pato/Skrivebord/HPV16_projekt/References/GFFfiles/"
mainfolder = "Organised_Fastas/"+maintype
e4fixSavePath = "/home/pato/Skrivebord/HPV16_projekt/References/E4fixCoords"

In [ ]:
# Checking that file does not already have AUHpred 
txt_file = open(gffFolder+"/"+subtype+".gff3", "r")
file_content = txt_file.read()
if "AUHpred" in file_content:
    sys.exit("File already have AUHpred")
else:
    print("updating file")

txt_file.close()

In [99]:
gfffile = gffFolder+subtype+".gff3"
#infile = pd.read_csv(gfffile)
with open(gfffile, "r") as file:
    gffstring = file.readlines()
    gffHeader1 = gffstring[0]
    gffHeader2 = gffstring[2]

# gffHeader1 = infile.columns[0]
# gffHeader2 = list(infile.loc[0])[0]

In [65]:
# # expanding file to create dataframe
# newlist = []

# for i in range(1,len(infile)):
#     newlist.append(list(infile.loc[i])[0])

In [66]:
# df = pd.DataFrame(newlist)
# fstSplit = df[0].str.split('\t', 8, expand=True)
# scnsplit = fstSplit[8].str.split(';', 8, expand=True)
# newdf = pd.concat([fstSplit,scnsplit], axis = 1)

In [67]:
# length = len(newdf.columns)
# for i in range(length):
#     newdf.columns.values[i] = i

In [68]:
# newdf

In [69]:
coords = pd.read_csv(mainfolder+"/"+subtype+"_E4coords.csv", header = None, sep = " ")

# checker for I og D i cigar:
if ("I" in coords.iloc[0,1]) | ("I" in coords.iloc[1,1]):
    sys.exit("Insertion found. Exiting...")
if ("D" in coords.iloc[0,1]) | ("D" in coords.iloc[1,1]):
    sys.exit("Deletion found. Exiting...")

e4s_1 = coords.iloc[0,0]
e4e_1 = e4s_1+ int((str(coords.iloc[0,1]).replace("M","")))
e4s_2 = coords.iloc[1,0]
e4e_2 = e4s_2+ int((str(coords.iloc[1,1]).replace("M","")))


In [110]:
# update the file
fs = gffHeader2.split("\t")
# Get the length of the first splice section to infer CDS phase. +1 because the positions are inclusive.
cdsPhaseCalc = (e4s_2 - e4s_1 + 1) % 3

if cdsPhaseCalc == 0:
    cdsPhase = 0
elif cdsPhaseCalc == 1:
    cdsPhase = 2
elif cdsPhaseCalc == 1:
    cdsPhase = 1
else:
    sys.exit("CDSphase does not go into 0,1 or 2")

row1 = [fs[0],"AUHpred","region",fs[3],fs[4],fs[5],fs[6],fs[7],fs[8]]
row2 = [fs[0],"AUHpred","gene",e4s_1,e4e_2,'.','+','.',"ID=E4_splice"]
row3 = [fs[0],"AUHpred","mRNA",e4s_1,e4e_2,'.','+','.',"ID=E4_spliceRNA;Parent=E4_splice"]
row4 = [fs[0],"AUHpred","CDS",e4s_1,e4e_1,'.','+','0',"ID=E4cds1;Parent=E4_spliceRNA"]
row5 = [fs[0],"AUHpred","CDS",e4s_2,e4e_2,'.','+',cdsPhase,"ID=E4cds2;Parent=E4_spliceRNA"]

list_lists=[row1,row2,row3,row4,row5]
spliceDf = pd.DataFrame(list_lists)
spliceDf.iloc[0,8] = spliceDf.iloc[0,8].split(":")[0]

spliceDf
# save to correct position

,0,1,2,3,4,5,6,7,8
0,FN907962_1,AUHpred,region,1,7934,.,+,.,ID=FN907962_1
1,FN907962_1,AUHpred,gene,832,3582,.,+,.,ID=E4_splice
2,FN907962_1,AUHpred,mRNA,832,3582,.,+,.,ID=E4_spliceRNA;Parent=E4_splice
3,FN907962_1,AUHpred,CDS,832,848,.,+,0,ID=E4cds1;Parent=E4_spliceRNA
4,FN907962_1,AUHpred,CDS,3325,3582,.,+,0,ID=E4cds2;Parent=E4_spliceRNA


In [74]:
# Add header and save to file

row0 = "##gff-version 3"
row01 = gffHeader1

file1 = open(e4fixSavePath+"/"+subtype+".gff3", "w")
file1.writelines(row0+"\n")
file1.writelines(row01+"\n")
file1.close()

spliceDf.to_csv(e4fixSavePath+"/"+subtype+".gff3", header = None, index = None, sep = "\t", mode = "a")

In [75]:
row1 = [fs[0],"AUHpred","gene",e4s_1,e4e_2,'.','+','.',"ID=E4_splice"]
row2 = [fs[0],"AUHpred","mRNA",e4s_1,e4e_1,'.','+','.',"ID=E4_spliceRNA1;Parent=E4_splice"]
row3 = [fs[0],"AUHpred","mRNA",e4s_2,e4e_2,'.','+','.',"ID=E4_spliceRNA2;Parent=E4_splice"]
row4 = [fs[0],"AUHpred","CDS",e4s_1,e4e_1,'.','+','0',"ID=E4exon1;Parent=E4_spliceRNA1"]
row5 = [fs[0],"AUHpred","CDS",e4s_2,e4e_2,'.','+',cdsPhase,"ID=E4exon2;Parent=E4_spliceRNA2"]

list_lists=[row1,row2,row3,row4,row5]
spliceDf = pd.DataFrame(list_lists)
spliceDf.iloc[0,8] = spliceDf.iloc[0,8].split(":")[0]

spliceDf.to_csv(gffFolder+"/"+subtype+".gff3", header = None, index = None, sep = "\t", mode = "a")